How to use gulp?

In [2]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-runtime');
    require.resolve('gulp-replace');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp#4.0" gulp-runtime gulp-replace gulp-tap');
}
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// rename the project
gulp.task('build :src :dest', function (files, project) {
    return gulp
        .src(files, {cwd: project})
        .pipe(replace('appId: \'my-app-id\'', 'appId: \'jupytangular-module-server\''))
        .pipe(tap(function(file) {
            console.log(file.contents.toString());
        }))
});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';

gulp.task('default', () => gulp.task(['build :src :dest'])(['**/app.browser.module.ts'], project));

$$.async()
try {
gulp.series(() => gulp.task(['default'])(), function (done) {
    done();
    $$.sendResult();
})();
} catch (e) {
    $$.sendError(e);
}

{ '0': [ '**/app.browser.module.ts' ],
  '1': '/Users/briancullinan/Documents/universal' }
// angular
import { NgModule } from '@angular/core';
import { BrowserModule } from '@angular/platform-browser';

// libs
import { BrowserStateTransferModule, DEFAULT_STATE_ID } from '@ngx-universal/state-transfer';
import { CACHE } from '@ngx-cache/core';
import { BrowserCacheModule, MemoryCacheService, STATE_ID } from '@ngx-cache/platform-browser';

// modules & components
import { AppModule } from './app.module';
import { AppComponent } from './app.component';
import { BrowserAnimationsModule } from '@angular/platform-browser/animations';
import { APP_BASE_HREF, PlatformLocation } from '@angular/common';
import { MockPlatformLocation } from './testing/location-mock';

@NgModule({
    bootstrap: [ AppComponent ],
    imports: [
        BrowserModule.withServerTransition({
            appId: 'jupytangular-module-server'
        }),
        BrowserStateTransferModule.forRoot(),
        BrowserCach

Apply universal to angular project?

In [3]:

var applyUniversal = (project) => {

    // add material theme?
    gulp.task('theme', function () {
        gulp.src('src/client/assets/sass')
            .pipe(replace(/^/ig, `
@import '~@angular/material/_theming';

@include mat-core();

$candy-app-primary: mat-palette($mat-indigo);
$candy-app-accent: mat-palette($mat-pink, A200, A100, A400);

// The warn palette is optional (defaults to red).
$candy-app-warn: mat-palette($mat-red);

// Create the theme object (a Sass map containing all of the palettes).
$theme: mat-light-theme($candy-app-primary, $candy-app-accent, $candy-app-warn);

@include angular-material-theme($theme);

`))
        .pipe(gulp.dest(project));
    });
    
    // add automock to project for callback services
    var renderer, server, core;
    var importer = gulp.task('importer', function () {
        return require('../Core')
            .interpretAll([
                'mock audit',
                'module promisifty socket.io',
                'promisifiy socket.io server',
                'sockify server running',
                'render Angular modules',
                'core modules'
            ])
            .then(r => {
                server = r[2].code;
                renderer = r[3].code;
                core = r[4].code;
                return r[0].code + r[1].code;
            });
    });
    gulp.task('sockifyclient', ['importer'], function () {
        gulp.src(['src/client/app/sockify-client.js'], {base: project})
            .pipe(importer())
            .pipe(gulp.dest(project))
    });
    
    // add shared module to app.module
    gulp.task('core', ['importer'], function () {
        gulp.src(['src/client/app/core.module.ts'], {base: project})
            .pipe(() => core)
            .pipe(gulp.dest(project))
    });

    // add server renderer to project
    gulp.task('renderer', ['importer'], function () {
        gulp.src(['src/server/render-server.ts'], {base: project})
            .pipe(() => renderer)
            .pipe(gulp.dest(project))
    });
    
    // tell webpack to build the renderer seperately
    gulp.task('webpackconfig', function () {
        gulp.src(['config/webpack.config.js'], {base: project})
            .pipe(replace('? webpackConfig.universal.server.prod(root, settings)', `
                    ? webpackMerge(webpackConfig.universal.server.prod(root, settings), {
                        entry: {
                            'render-service': root(\`${settings.paths.src.server.root}/render-service.ts\`),
                            'server': root(\`${settings.paths.src.server.root}/server.ts\`)
                        },
                        output: {
                            filename: '[name].js'
                        }
                    })
`))
            .pipe(gulp.dest(project))
    });
    
    // replace app module module with these?
    gulp.task('appmodule', function(){
      gulp.src(['src/client/app.module.ts'], {base: project})
        // add some dependencies
        .pipe(replace('import { AppComponent } from \'./app.component\';', `
import { AppComponent } from './app.component';
import { SharedModule, sharedModules } from './core.module';
import { Observable } from 'rxjs/Observable';
import { ConfigFsLoader } from '@ngx-config/fs-loader';
`))
        // replace the configuration loader with a Mock
        .pipe(replace(
          'const browserLoader = new ConfigHttpLoader(http, \'./assets/config.json\')',
          `
    const browserLoader = <ConfigHttpLoader>{
        loadSettings: () => Promise.resolve(
            (typeof window !== 'undefined' ? <any>window : {}).CONFIG
            || {})
    };
`))
        // replace router
        .pipe(replace(
          'RouterModule.forRoot(routes),',
          `
        RouterModule.forRoot(routes, {initialNavigation: 'enabled', useHash: false}),
`))
        // don't care about meta for this
        .pipe(replace(/MetaLoader \{[\s\S]*?\});/ig,
          `MetaLoader {
        return new MetaStaticLoader();
`))
        .pipe(replace(/TranslateLoader \{[\s\S]*?\});/ig, `
    return <TranslateLoader>{
        getTranslation: lang => Observable.of(
            (typeof window !== 'undefined' ? <any>window : {}).TRANSLATIONS)
    };
`))
        .pipe(gulp.dest(project));
    });
    
    // add socket-client provider for auth module to app.browser.module
    gulp.task('browsermodule', function(){
      gulp.src(['src/client/app.browser.module.ts'], {base: project})
        .pipe(replace(
         'AppModule',
          `AppModule,
        BrowserAnimationsModule,
    ],
    providers: [
        {
            provide: AuthService,
            useValue: require('./sockify-client').sockifyClient(AuthService, 'AuthService'),
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-client').sockifyClient(PlatformLocation, 'PlatformLocation'),
        },
`))
        .pipe(gulp.dest(project));
    });

    // replace app component
    gulp.task('appcomponent', function(){
      gulp.src(['src/client/app.component.html'], {base: project})
        .pipe(replace(
         /[\s\S].*/ig, '<router-outlet></router-outlet>'))
        .pipe(gulp.dest(project));
    });
    
    // replace routes with lazy modules from other projects
    gulp.task('approuter', function(){
      gulp.src(['src/client/app.component.html'], {base: project})
        .pipe(replace(
         /Routes = \[[\s\S].*?];/ig, `
    {
        path: '',
        children: [
            {
                path: 'auth',
                loadChildren: './auth/auth.module#AuthModule'
            }
        ],
        canActivateChild: [ MetaGuard ],
        data: {
            i18n: {
                isRoot: true
            }
        }
    },
    {
        path: 'change-language/:languageCode',
        component: ChangeLanguageComponent
    },
    {
        path: '**',
        redirectTo: '',
        pathMatch: 'full'
    }
`))
        .pipe(gulp.dest(project));
    });

    // modify server
    gulp.task('server', function() {
      gulp.src(['src/server/server.ts'], {base: project})
        // remove this silly message?
        .pipe(replace('console.log(\'bootstrapFactory\');', ''))
        // add mock socket server to server/render-server or app.server.module
        .pipe(replace(
         'provide: APP_BASE_HREF,',
          `
                                provide: AuthService,
                                useValue: require('./sockify-server').sockifyRequire(AuthService, 'AuthService')
                            },
                            {
                                provide: PlatformLocation,
                                useValue: require('./sockify-server').sockifyRequire(PlatformLocation, 'PlatformLocation')
                            },
                            {
`))
        .pipe(gulp.dest(project));
    });
        
    // add server mock reporter to project
    gulp.task('sockifyserver', ['importer'], function () {
        gulp.src(['src/server/sockify-server.ts'], {base: project})
            .pipe(() => server)
            .pipe(gulp.dest(project))
    });
    
    // add components files from other projects
    
    
    // gulp apply everything
    gulp.task('applyuniversal', function () {
        gulp.series('theme',
                   'importer',
                   'sockifyclient',
                   'core',
                   'renderer',
                   'webpackconfig',
                   'appmodule',
                   'browsermodule',
                   'appcomponent',
                   'server',
                   'sockifyserver');
    });

    return gulp.task('applyuniversal');

};
(applyUniversal);

[Function: applyUniversal]

In [ ]:

$$.async()
applyUniversal(project)
    .on('end', r => $$.sendResult(r))
    .on('error', e => $$.sendError(e));

